In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ninadaithal/imagesoasis")

print("Path to dataset files:", path)

c:\Users\rohan\Desktop\184a\AD-Detection\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.23G/1.23G [00:50<00:00, 26.0MB/s]

Extracting files...


Path to dataset files: C:\Users\rohan\.cache\kagglehub\datasets\ninadaithal\imagesoasis\versions\1


In [2]:
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# Create your DataFrame
images = []
labels = []
for subfolder in tqdm(os.listdir(r'C:\Users\rohan\Desktop\184a\AD-Detection\archive')):
    subfolder_path = os.path.join(r'C:\Users\rohan\Desktop\184a\AD-Detection\archive', subfolder)
    label = 0
    for folder in os.listdir(subfolder_path):
        subfolder_path2 = os.path.join(subfolder_path, folder)
        for image_filename in os.listdir(subfolder_path2):
            image_path = os.path.join(subfolder_path2, image_filename)
            images.append(image_path)
            labels.append(label)
        label += 1

df = pd.DataFrame({'image': images, 'label': labels})
df

100%|██████████| 1/1 [00:00<00:00,  3.08it/s]


,image,label
0,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
1,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
2,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
3,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
4,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
...,...,...
86432,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
86433,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
86434,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
86435,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3


In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], shuffle=True, random_state=42)

In [5]:
class ADDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing image paths and labels.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get image path and label
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label



# Define transformations
transform = transforms.Compose([
    transforms.Resize((176, 176)), # Resize all images to 176x176
    transforms.ToTensor(),        # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with ImageNet stats
])

In [6]:
# Create Datasets for training and testing
train_dataset = ADDataset(dataframe=train_df, transform=transform)
test_dataset = ADDataset(dataframe=test_df, transform=transform)

# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Test the DataLoader
print("Training DataLoader:")
for images, labels in train_dataloader:
    print(images.shape)  # Should be [batch_size, 3, 176, 176]
    print(labels)        # Corresponding labels
    break

print("Testing DataLoader:")
for images, labels in test_dataloader:
    print(images.shape)  # Should be [batch_size, 3, 176, 176]
    print(labels)        # Corresponding labels
    break

Training DataLoader:
torch.Size([16, 3, 176, 176])
tensor([2, 2, 2, 2, 2, 3, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2])
Testing DataLoader:
torch.Size([16, 3, 176, 176])
tensor([2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


In [7]:
class FeatureExtractionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FeatureExtractionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.batch_norm(x)
        x = self.max_pool(x)
        return x


class ADCNN(nn.Module):
    def __init__(self, dropout_rate=0.25):
        super(ADCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Feature Extraction Blocks
        self.block1 = FeatureExtractionBlock(16, 32)  # Output: 32 channels
        self.block2 = FeatureExtractionBlock(32, 64)  # Output: 64 channels
        self.block3 = FeatureExtractionBlock(64, 128)  # Output: 128 channels
        self.block4 = FeatureExtractionBlock(128, 256)  # Output: 256 channels

        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64) 
        self.fc4 = nn.Linear(64, 4)
        
        # Softmax layer is applied in forward
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.dropout1(x)
        x = self.block4(x)
        x = self.dropout2(x)

        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        #x = self.softmax(x)
        return x

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = ADCNN()

cuda:0


In [9]:

for inputs, labels in train_dataloader:
    print(inputs.shape, labels.shape)
    break

def train_model(model, train_loader, num_epochs=50, learning_rate=0.001, device='cuda'):
    # Move model to the selected device (GPU or CPU)
    model.to(device)
    
    # Initialize RMSprop optimizer
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    
    # Loss function (CrossEntropy for classification)
    criterion = torch.nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Move data to the selected device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Backward pass (backpropagation)
            loss.backward()

            # Update weights
            optimizer.step()

            # Track running loss
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Print stats for the epoch
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

        # Optional: You can validate your model here by running the validation loop if you have a validation set.
        # validate_model(model, val_loader)  # Add your validation code here
     
    print('Training Finished!')


train_model(model, train_dataloader , num_epochs=50, learning_rate=0.001, device='cuda')

torch.Size([16, 3, 176, 176]) torch.Size([16])
Epoch [1/50], Loss: 0.8516, Accuracy: 77.29%
Epoch [2/50], Loss: 0.3140, Accuracy: 87.46%
Epoch [3/50], Loss: 0.1346, Accuracy: 95.06%
Epoch [4/50], Loss: 0.0755, Accuracy: 97.44%
Epoch [5/50], Loss: 0.0529, Accuracy: 98.22%
Epoch [6/50], Loss: 0.0406, Accuracy: 98.68%
Epoch [7/50], Loss: 0.0336, Accuracy: 98.88%
Epoch [8/50], Loss: 0.0287, Accuracy: 99.12%
Epoch [9/50], Loss: 0.0254, Accuracy: 99.24%
Epoch [10/50], Loss: 0.0245, Accuracy: 99.27%
Epoch [11/50], Loss: 0.0232, Accuracy: 99.34%
Epoch [12/50], Loss: 0.0229, Accuracy: 99.43%
Epoch [13/50], Loss: 0.0191, Accuracy: 99.46%
Epoch [14/50], Loss: 0.0180, Accuracy: 99.51%
Epoch [15/50], Loss: 0.0176, Accuracy: 99.56%
Epoch [16/50], Loss: 0.0179, Accuracy: 99.55%
Epoch [17/50], Loss: 0.0164, Accuracy: 99.56%
Epoch [18/50], Loss: 0.0155, Accuracy: 99.58%
Epoch [19/50], Loss: 0.0145, Accuracy: 99.62%
Epoch [20/50], Loss: 0.0176, Accuracy: 99.70%
Epoch [21/50], Loss: 0.0138, Accuracy: 99.

In [10]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import itertools
import torch

def evaluate_model(model, test_loader, device='cuda'):
    y_pred_list = []
    y_target_list = []

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No gradient calculations needed
        for inputs, labels in test_loader:
            # Move data to the selected device
            inputs, labels = inputs.to(device), labels.to(device)

            # Get model predictions
            outputs = model(inputs)
            _, y_pred = torch.max(outputs, 1)  # Predicted labels
            
            y_pred_list.append(y_pred.cpu().numpy())  # Move to CPU for metrics
            y_target_list.append(labels.cpu().numpy())  # Move to CPU for metrics

    # Flatten the predictions and targets
    y_pred_list = list(itertools.chain.from_iterable(y_pred_list))
    y_target_list = list(itertools.chain.from_iterable(y_target_list))

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_target_list, y_pred_list)
    print("Confusion Matrix of the Test Set")
    print("-----------")
    print(conf_matrix)

    # Calculate metrics
    precision = precision_score(y_target_list, y_pred_list, average='weighted')
    recall = recall_score(y_target_list, y_pred_list, average='weighted')
    f1 = f1_score(y_target_list, y_pred_list, average='weighted')

    print(f"Precision of the Model :\t{precision:.4f}")
    print(f"Recall of the Model    :\t{recall:.4f}")
    print(f"F1 Score of the Model  :\t{f1:.4f}")

# Call the evaluation function
evaluate_model(model, test_dataloader, device='cuda')


Confusion Matrix of the Test Set
-----------
[[ 1000     0     0     0]
 [    0    98     0     0]
 [    0     0 13442     3]
 [    0     0     1  2744]]
Precision of the Model :	0.9998
Recall of the Model    :	0.9998
F1 Score of the Model  :	0.9998
